In [1]:
!python3 -m venv bug-detection
!source bug-detection/bin/activate
!pip install torch transformers datasets scikit-learn flake8

Error: Command '['/content/bug-detection/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: bug-detection/bin/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 

In [2]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

def preprocess_code(code_snippet):
    return tokenizer(code_snippet, return_tensors='pt', padding=True, truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [3]:
import pandas as pd
import random

# Define categories for NLP classification (Example: Sentiment Analysis - Positive, Negative, Neutral)
categories = ["Positive", "Negative", "Neutral"]

# Sample text data for NLP
data_samples = [
    "I absolutely love this product! It's fantastic!",
    "This is the worst experience I have ever had.",
    "The service was okay, not the best but not the worst.",
    "Amazing quality! I will buy it again for sure.",
    "I regret purchasing this. It was a complete waste of money.",
    "Not sure how I feel about this, it's just average.",
    "Excellent performance and great customer support!",
    "Terrible experience, just terrible. Would not recommend.",
    "It was fine, nothing special but not bad either.",
    "Highly recommended! A wonderful and delightful experience."
]

# Generate dataset with random labels
dataset = {"text": data_samples, "label": [random.choice(categories) for _ in data_samples]}
df = pd.DataFrame(dataset)

# Save dataset to CSV file
df.to_csv("train_dataset.csv", index=False)

print("Dataset created successfully!")
print(df.head())


Dataset created successfully!
                                                text     label
0    I absolutely love this product! It's fantastic!  Positive
1      This is the worst experience I have ever had.  Negative
2  The service was okay, not the best but not the...  Negative
3     Amazing quality! I will buy it again for sure.   Neutral
4  I regret purchasing this. It was a complete wa...  Negative


In [4]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import pandas as pd
from datasets import Dataset, DatasetDict

# ... (your previous code) ...

# Load the dataset from the CSV file
train_dataset_df = pd.read_csv("train_dataset.csv")

# Define a mapping from string labels to numerical IDs
label2id = {"Positive": 0, "Negative": 1, "Neutral": 2}
id2label = {0: "Positive", 1: "Negative", 2: "Neutral"}  # For inverse mapping if needed

# Convert string labels to numerical IDs
train_dataset_df['label'] = train_dataset_df['label'].map(label2id)

# Convert the pandas DataFrame to a Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_dataset_df)

# Define a preprocessing function for individual examples
def preprocess_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

# Apply the preprocessing function to the dataset
tokenized_train = train_dataset.map(preprocess_function, batched=True)

# Define data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define the model with the correct number of labels
model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=len(label2id))


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,  # Add the data collator
    # Remove eval_dataset if you don't have a separate evaluation set
    # eval_dataset=tokenized_eval,
)

trainer.train()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sneha2018das (sneha2018das-brainware-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=6, training_loss=1.0591511726379395, metrics={'train_runtime': 223.9172, 'train_samples_per_second': 0.134, 'train_steps_per_second': 0.027, 'total_flos': 7893402531840.0, 'train_loss': 1.0591511726379395, 'epoch': 3.0})

In [5]:
def detect_bug_in_code(code_snippet):
    inputs = preprocess_code(code_snippet)
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits)
    if predicted_label == 1:
        return True  # Bug detected
    else:
        return False  # No bug detected

In [6]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

def fix_bug(code_snippet):
    model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base")
    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=512)
    fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code

In [12]:
import argparse

def detect_bug_in_code(code_snippet):
    # Placeholder function: Implement bug detection logic
    return "bug" in code_snippet.lower()  # Example: detecting if the word "bug" exists

def fix_bug(code_snippet):
    # Placeholder function: Implement bug fixing logic
    return code_snippet.replace("bug", "fixed")  # Example: replacing "bug" with "fix"

def main(code_snippet="This code has bug"): # Added a default value for code_snippet
    """
    Detects and fixes bugs in a given code snippet.

    Args:
        code_snippet (str): The code snippet to analyze. Defaults to "This code has bug".
    """
    # parser = argparse.ArgumentParser(description="Bug detection and fixing")
    # parser.add_argument('code_snippet', type=str, help="Buggy code to analyze and fix")
    # args = parser.parse_args()

    if detect_bug_in_code(code_snippet):
        print("Bug detected!")
        fixed_code = fix_bug(code_snippet)
        print(f"Fix code:\n{fixed_code}")
    else:
        print("No bug detected.")

if __name__ == "__main__":
    main()

Bug detected!
Fix code:
This code has fixed
